In [ ]:
def backtest(data, sl, tp, n_shares, trial) -> float:
    # --- Copia del dataset ---
    data = data.copy()

    # --- Capital inicial ---
    capital = BacktestingCapCOM.initial_capital
    cash = float(capital)

    # --- Hiperparámetros ---
    params = hyperparams(trial)
    rsi_window = params["rsi_window"]
    rsi_lower = params["rsi_lower"]
    rsi_upper = params["rsi_upper"]
    momentum_window = params["momentum_window"]
    momentum_threshold = params["momentum_threshold"]
    volatility_window = params["volatility_window"]
    volatility_threshold = params["volatility_threshold"]
    sl = params["stop_loss"]
    tp = params["take_profit"]
    n_shares = params["n_shares"]

    COM = BacktestingCapCOM.COM

    # --- Señales ---
    buy_rsi, sell_rsi = Indicadores.get_rsi(
        data, rsi_window, rsi_upper, rsi_lower)
    buy_momentum, sell_momentum = Indicadores.get_momentum(
        data, momentum_window, momentum_threshold)
    buy_volatility, sell_volatility = Indicadores.get_volatility(
        data, volatility_window, volatility_threshold)

    data["buy_signal"] = buy_rsi & buy_momentum & buy_volatility
    data["sell_signal"] = sell_rsi & sell_momentum & sell_volatility

    # --- Posiciones activas ---
    active_long_positions: list[Position] = []
    active_short_positions: list[Position] = []

    # --- Historial del portafolio ---
    port_hist = []
    portafolio_value = cash

    # --- Iterar el DataFrame ---
    for i, row in data.iterrows():

        # === CIERRE DE POSICIONES ===
        for pos in active_long_positions.copy():
            if (pos.sl > row.Close) or (pos.tp < row.Close):
                cash += row.Close * pos.n_shares * (1 - COM)
                active_long_positions.remove(pos)

        for pos in active_short_positions.copy():
            for pos in active_short_positions.copy():
                if (pos.sl < row.Close) or (pos.tp > row.Close):
                    cash += (pos.price * pos.n_shares) + \
                        (pos.price - row.Close) * pos.n_shares * (1 - COM)
                    active_short_positions.remove(pos)

        # === APERTURA DE POSICIONES ===
        if row.sell_signal:  # Entrada SHORT
            cost = row.Close * n_shares * (1 + COM)
            if cash >= cost:
                cash -= cost
                active_short_positions.append(Position(
                    price=row.Close,
                    n_shares=n_shares,
                    sl=row.Close * (1 + sl),
                    tp=row.Close * (1 - tp),
                ))

        if row.buy_signal:  # Entrada LONG
            cost = row.Close * n_shares * (1 + COM)
            if cash >= cost:
                cash -= cost
                active_long_positions.append(Position(
                    price=row.Close,
                    n_shares=n_shares,
                    sl=row.Close * (1 - sl),
                    tp=row.Close * (1 + tp),
                ))

        # === VALOR DEL PORTAFOLIO ===
        portafolio_value = cash

        for pos in active_long_positions:
            portafolio_value += row.Close * pos.n_shares

        for pos in active_short_positions:
            portafolio_value += (pos.price * pos.n_shares) + \
                (pos.price - row.Close) * pos.n_shares

        port_hist.append(portafolio_value)

    # --- Retorno final ---
    return port_hist



In [ ]:
def backtest(data, trial) -> tuple[list, dict, float]:
    """
    Executes a backtest on the provided dataset using specified hyperparameters.

    Parameters:
    -----------
    data : pd.DataFrame
        Historical price data, must include a 'Close' column.
    trial : optuna.trial.Trial
        Optuna trial object to suggest hyperparameters.

    Returns:
    --------
    port_hist : list[float]
        History of portfolio values at each timestep.
    metrics : dict
        Dictionary of performance metrics including Sharpe, Sortino, Win Rate, Max Drawdown, Calmar, 
        and Win Rate on Long Positions.
    cash : float
        Final available cash at the end of the backtest.

    Notes:
    ------
    The function:
    1. Initializes capital and commission.
    2. Retrieves hyperparameters from Optuna trial.
    3. Generates buy/sell signals based on RSI, momentum, and volatility indicators.
    4. Iterates over the data to open and close positions according to signals.
    5. Computes portfolio value at each step.
    6. Calculates final metrics based on closed trades and portfolio returns.
    """
    data = data.copy()

    # --- Initial capital ---
    capital = BacktestingCapCOM.initial_capital
    cash = float(capital)

    # --- Hyperparameters ---
    params = hyperparams(trial)
    rsi_window = params["rsi_window"]
    rsi_lower = params["rsi_lower"]
    rsi_upper = params["rsi_upper"]
    momentum_window = params["momentum_window"]
    momentum_threshold = params["momentum_threshold"]
    volatility_window = params["volatility_window"]
    volatility_threshold = params["volatility_threshold"]
    sl = params["stop_loss"]
    tp = params["take_profit"]
    n_shares = params["n_shares"]

    COM = BacktestingCapCOM.COM
# --- Generate signals ---
    buy_rsi, sell_rsi = Indicadores.get_rsi(
        data, rsi_window, rsi_upper, rsi_lower
    )
    buy_momentum, sell_momentum = Indicadores.get_momentum(
        data, momentum_window, momentum_threshold
    )
    buy_volatility, sell_volatility = Indicadores.get_volatility(
        data, volatility_window, volatility_threshold
    )

    # Guardamos cada señal en el DataFrame
    data["rsi_buy"] = buy_rsi
    data["rsi_sell"] = sell_rsi
    data["momentum_buy"] = buy_momentum
    data["momentum_sell"] = sell_momentum
    data["volatility_buy"] = buy_volatility
    data["volatility_sell"] = sell_volatility

    # Señal final: al menos 2 de 3
    data["buy_signal"] = (data[["rsi_buy", "momentum_buy", "volatility_buy"]].sum(axis=1) >= 2)
    data["sell_signal"] = (data[["rsi_sell", "momentum_sell", "volatility_sell"]].sum(axis=1) >= 2)

    # --- DEBUG: number of signals ---
    print("Active BUY signals:", data["buy_signal"].sum())
    print("Active SELL signals:", data["sell_signal"].sum())

    # Drop rows with NaN signals
    data = data.dropna(
        subset=[
            'rsi_buy', 'rsi_sell',
            'momentum_buy', 'momentum_sell',
            'volatility_buy', 'volatility_sell'
        ]
    ).reset_index(drop=True)

    # --- Active and closed positions ---
    active_long_positions: list[Position] = []
    active_short_positions: list[Position] = []
    closed_long_positions: list[Position] = []
    closed_short_positions: list[Position] = []

    # --- Portfolio history ---
    port_hist = [cash]

    # --- Iterate over dataset ---
    for idx, row in data.iterrows():
        price = row.Close

        # === CLOSE POSITIONS ===
        for pos in active_long_positions.copy():
            if price <= pos.sl or price >= pos.tp:
                cash += price * pos.n_shares * (1 - COM)
                pos.exit_price = price
                pos.profit = (price - pos.price) * pos.n_shares
                closed_long_positions.append(pos)
                active_long_positions.remove(pos)



        for pos in active_short_positions.copy():
            if price >= pos.sl or price <= pos.tp:
                pnl = (pos.price - price) * pos.n_shares * (1 - COM)
                cash += (pos.price * pos.n_shares) + pnl
                pos.exit_price = price
                pos.profit = (pos.price - price) * pos.n_shares
                closed_short_positions.append(pos)
                active_short_positions.remove(pos)

        # === OPEN POSITIONS ===
        if row.buy_signal:  # LONG
            cost = price * n_shares * (1 + COM)
            if cash >= cost:
                cash -= cost
                active_long_positions.append(Position(
                    price=price,
                    n_shares=n_shares,
                    sl=price * (1 - sl),
                    tp=price * (1 + tp),
                ))

        if row.sell_signal:  # SHORT
            cost = price * n_shares * (1 + COM)
            if cash >= cost:
                cash -= cost
                active_short_positions.append(Position(
                    price=price,
                    n_shares=n_shares,
                    sl=price * (1 + sl),
                    tp=price * (1 - tp),
                ))

        # === PORTFOLIO VALUE ===
        port_value = cash
        for pos in active_long_positions:
            port_value += price * pos.n_shares
        for pos in active_short_positions:
            port_value += (pos.price * pos.n_shares) + \
                (pos.price - price) * pos.n_shares
        port_hist.append(port_value)

    # === METRICS (final) ===
    df = pd.DataFrame({'PortValue': port_hist})
    df['Returns'] = df.PortValue.pct_change().fillna(0)

    # Extract profits from closed positions
    long_profits = [
        pos.profit for pos in closed_long_positions if pos.profit is not None]
    short_profits = [
        pos.profit for pos in closed_short_positions if pos.profit is not None]

    metrics = {
        'Sharpe': Metrics(df.Returns).sharpe,
        'Sortino': Metrics(df.Returns).sortino,
        'Win Rate': sum(p > 0 for p in long_profits + short_profits) / max(1, len(long_profits + short_profits)),
        'Max Drawdown': Metrics(df.Returns).max_drawdown,
        'Calmar': Metrics(df.Returns).calmar,
        'Win Rate on Long Positions': sum(p > 0 for p in long_profits) / max(1, len(long_profits))
    }

    # --- DEBUG: closed positions ---
    print("Closed LONG positions:", len(closed_long_positions))
    print("Closed SHORT positions:", len(closed_short_positions))

    return port_hist, metrics, cash


In [ ]:
from libraries import *
from funtions import Position, BacktestingCapCOM
from Objetive import hyperparams
from Indicadores import Indicadores
from metrics import Metrics


def backtest(data, trial) -> float:
    """
    Backtest con lógica similar a backtest_opt:
    - Señales: RSI, Momentum, Volatilidad (al menos 2 de 3).
    - Posiciones: tamaño dinámico (porcentaje del capital).
    - TP/SL por operación.
    - Cierre de posiciones al final del periodo.
    - Retorna Calmar ratio como métrica de optimización.
    """
    data = data.copy().reset_index(drop=True)

    # --- Hyperparams de Optuna ---
    params = hyperparams(trial)
    rsi_window = params["rsi_window"]
    rsi_lower = params["rsi_lower"]
    rsi_upper = params["rsi_upper"]

    momentum_window = params["momentum_window"]
    momentum_threshold = params["momentum_threshold"]

    volatility_window = params["volatility_window"]
    volatility_threshold = params["volatility_threshold"]

    stop_loss = params["stop_loss"]
    take_profit = params["take_profit"]
    capital_pct_exp = trial.suggest_float("capital_pct_exp", 0.01, 0.20)

    # --- Comisión y capital inicial ---
    COM = BacktestingCapCOM.COM
    cash = BacktestingCapCOM.initial_capital

    # --- Señales por indicador ---
    buy_rsi, sell_rsi = Indicadores.get_rsi(data, rsi_window, rsi_upper, rsi_lower)
    buy_momentum, sell_momentum = Indicadores.get_momentum(data, momentum_window, momentum_threshold)
    buy_volatility, sell_volatility = Indicadores.get_volatility(data, volatility_window, volatility_threshold)

    # --- Combinar señales ---
    historic = data.copy()
    historic["buy_signal"] = (
        buy_rsi.astype(int) + buy_momentum.astype(int) + buy_volatility.astype(int)
    ) >= 2
    historic["sell_signal"] = (
        sell_rsi.astype(int) + sell_momentum.astype(int) + sell_volatility.astype(int)
    ) >= 2

    historic = historic.dropna().reset_index(drop=True)

    # --- Posiciones ---
    active_long_positions: list[Position] = []
    active_short_positions: list[Position] = []
    port_value = []

    # --- Iteración principal ---
    for i, row in historic.iterrows():
        price = row.Close
        n_shares = (cash * capital_pct_exp) / price

        # === Cerrar LONG ===
        for pos in active_long_positions.copy():
            if price >= pos.tp or price <= pos.sl:
                cash += price * pos.n_shares * (1 - COM)
                active_long_positions.remove(pos)

        # === Cerrar SHORT ===
        for pos in active_short_positions.copy():
            if price <= pos.tp or price >= pos.sl:
                pnl = (pos.price - price) * pos.n_shares * (1 - COM)
                cash += (pos.price * pos.n_shares * (1 + COM)) + pnl
                active_short_positions.remove(pos)

        # === Abrir LONG ===
        if row.buy_signal and cash > price * n_shares * (1 + COM):
            cash -= price * n_shares * (1 + COM)
            active_long_positions.append(
                Position(
                    price=price,
                    n_shares=n_shares,
                    sl=price * (1 - stop_loss),
                    tp=price * (1 + take_profit),
                )
            )

        # === Abrir SHORT ===
        if row.sell_signal and cash > price * n_shares * (1 + COM):
            cash -= price * n_shares * (1 + COM)
            active_short_positions.append(
                Position(
                    price=price,
                    n_shares=n_shares,
                    sl=price * (1 + stop_loss),
                    tp=price * (1 - take_profit),
                )
            )

        # Valor del portafolio en cada paso
        port_value.append(
            cash
            + sum(price * pos.n_shares for pos in active_long_positions)
            + sum((pos.price * pos.n_shares) + (pos.price - price) * pos.n_shares for pos in active_short_positions)
        )

    # === Cerrar posiciones al final ===
    for pos in active_long_positions:
        cash += price * pos.n_shares * (1 - COM)

    for pos in active_short_positions:
        pnl = (pos.price - price) * pos.n_shares * (1 - COM)
        cash += (pos.price * pos.n_shares * (1 + COM)) + pnl

    # --- Métrica: Calmar Ratio ---
# Al final de tu función backtest
    df = pd.DataFrame({"PortValue": port_value})
    calmar = Metrics(df.PortValue).calmar

    # Retornar historial de portafolio, diccionario de métricas y cash final
    metrics_dict = {"Calmar": calmar}
    return port_value, metrics_dict, cash
